In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packa

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-08 19:16:55--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-06-08 19:16:55 (6.44 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
# Import the file from S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Create the vine_df DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
# Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_gt20_df = vine_df.filter("total_votes >= 20")
vine_gt20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [8]:
vine_gt20_df.count()

65379

In [9]:
# Filter the new DataFrame created to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%

vine_filtered = vine_gt20_df.filter("helpful_votes/total_votes >= 0.5")
vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [10]:
vine_filtered.count()

40565

In [11]:
# Filter the DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_paid = vine_filtered.filter(vine_filtered["vine"] == 'Y')
vine_paid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|     

In [12]:
vine_paid.count()

94

In [13]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

vine_unpaid = vine_filtered.filter(vine_filtered["vine"] == 'N')
vine_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [14]:
vine_unpaid.count()

40471

In [39]:
# Total number of Paid Vine reviews

ttl_vine_paid_reviews = vine_paid.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Total Paid Vine Reviews")
ttl_vine_paid_reviews.show()

+-----------------------+
|Total Paid Vine Reviews|
+-----------------------+
|                     94|
+-----------------------+



In [38]:
# Total number of 5-Star Paid Vine reviews

vine_paid_5star_reviews = vine_paid.filter("star_rating == 5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "Paid Vine 5 Star Reviews")
vine_paid_5star_reviews.show()

+------------------------+
|Paid Vine 5 Star Reviews|
+------------------------+
|                      48|
+------------------------+



In [60]:
# Calculate the percentage of Paid vine 5-star reviews
ttl_vine_paid_percent_reviews = vine_paid_5star_reviews.collect()[0]['Paid Vine 5 Star Reviews']/ttl_vine_paid_reviews.collect()[0]['Total Paid Vine Reviews'] * 100

In [61]:
# Rounding the value to 2 decimal places
round(ttl_vine_paid_percent_reviews,2)

51.06

In [63]:
# Total number of UnPaid Vine reviews
ttl_vine_unpaid_reviews = vine_unpaid.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Total Unpaid Vine Reviews")
ttl_vine_unpaid_reviews.show()

+-------------------------+
|Total Unpaid Vine Reviews|
+-------------------------+
|                    40471|
+-------------------------+



In [64]:
# Total number of 5-Star UnPaid Vine reviews
vine_unpaid_5star_reviews = vine_unpaid.filter("star_rating == 5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "Unpaid Vine 5 Star Reviews")
vine_unpaid_5star_reviews.show()

+--------------------------+
|Unpaid Vine 5 Star Reviews|
+--------------------------+
|                     15663|
+--------------------------+



In [65]:
# Calculate the percentage of Paid vine 5-star reviews
ttl_vine_unpaid_percent_reviews = vine_unpaid_5star_reviews.collect()[0]['Unpaid Vine 5 Star Reviews']/ttl_vine_unpaid_reviews.collect()[0]['Total Unpaid Vine Reviews'] * 100
round(ttl_vine_unpaid_percent_reviews,2)

38.7

In [66]:
# Create data frame for % results
percentage_df = spark.createDataFrame([
                                       (51.06, 38.7)],
                                      ['Percentage Vine Reviews', "Percentage non-Vine Reviews"])
percentage_df.show()

+-----------------------+---------------------------+
|Percentage Vine Reviews|Percentage non-Vine Reviews|
+-----------------------+---------------------------+
|                  51.06|                       38.7|
+-----------------------+---------------------------+



In [68]:
vine_paid.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [73]:
vine_paid_ratings = vine_paid.groupBy("star_rating").count().withColumnRenamed("count","Count of Vine Reviews").orderBy(vine_paid["star_rating"])
vine_paid_ratings.show()

+-----------+---------------------+
|star_rating|Count of Vine Reviews|
+-----------+---------------------+
|          1|                    1|
|          2|                    5|
|          3|                   16|
|          4|                   24|
|          5|                   48|
+-----------+---------------------+



In [77]:
import pyspark.sql.functions as func
vine_paid_ratings_percent = vine_paid_ratings.withColumn("% of Total", func.round((vine_paid_ratings['Count of Vine Reviews']/ttl_vine_paid_reviews.collect()[0]['Total Paid Vine Reviews']*100),2))
vine_paid_ratings_percent.show()

+-----------+---------------------+----------+
|star_rating|Count of Vine Reviews|% of Total|
+-----------+---------------------+----------+
|          1|                    1|      1.06|
|          2|                    5|      5.32|
|          3|                   16|     17.02|
|          4|                   24|     25.53|
|          5|                   48|     51.06|
+-----------+---------------------+----------+



In [74]:
vine_unpaid_ratings = vine_unpaid.groupBy("star_rating").count().withColumnRenamed("count","Count of Unpaid Vine Reviews").orderBy(vine_unpaid["star_rating"])
vine_unpaid_ratings.show()

+-----------+----------------------------+
|star_rating|Count of Unpaid Vine Reviews|
+-----------+----------------------------+
|          1|                       10303|
|          2|                        3388|
|          3|                        4379|
|          4|                        6738|
|          5|                       15663|
+-----------+----------------------------+



In [78]:
vine_unpaid_ratings_percent = vine_unpaid_ratings.withColumn("% of Total", func.round((vine_unpaid_ratings['Count of Unpaid Vine Reviews']/ttl_vine_unpaid_reviews.collect()[0]['Total Unpaid Vine Reviews'] * 100),2))
vine_unpaid_ratings_percent.show()

+-----------+----------------------------+----------+
|star_rating|Count of Unpaid Vine Reviews|% of Total|
+-----------+----------------------------+----------+
|          1|                       10303|     25.46|
|          2|                        3388|      8.37|
|          3|                        4379|     10.82|
|          4|                        6738|     16.65|
|          5|                       15663|      38.7|
+-----------+----------------------------+----------+

